In [2]:



from typing import Tuple
import numpy as np
import numpy.typing as npt
from scipy.signal import hilbert

def get_delay(phase: npt.NDArray) -> int:
    phase = phase
    m, n = phase.shape
    c1 = n * m
    r_phase = np.roll(phase, 1, axis=1)
    phase_product = np.multiply(phase, r_phase)
    c2 = (phase_product < 0).sum()
    delay = int(np.round(c1 / c2))
    print(f"delay = ",delay)
    return delay

def get_phase(time_series: npt.ArrayLike) -> npt.NDArray:
    complex_series = hilbert(time_series, axis=1)
    phase = np.angle(complex_series)
    return phase

def get_discretized_phase(phase: npt.NDArray, binsize: float) -> npt.NDArray:
    d_phase = np.ceil(phase / binsize).astype(np.int32)
    return d_phase

def get_binsize(phase: npt.NDArray, c: float = 3.49) -> float:
    m, n = phase.shape
    binsize = c * np.mean(np.std(phase, axis=1, ddof=1)) * n ** (-1.0 / 3)
    return binsize

def get_bincount(binsize: float) -> int:
    bins_w = np.arange(0, 2 * np.pi, binsize)
    bincount = len(bins_w)
    return bincount

import numpy as np
from scipy.signal import hilbert
from typing import Tuple
import numpy.typing as npt

def compute_PTE(phase: npt.NDArray, delay: int) -> npt.NDArray:
    m, n = phase.shape
    PTE = np.zeros((m, m), dtype=float)
    eps = 1e-12  # small constant to avoid log(0)

    length = n - delay

    for i in range(m):
        for j in range(m):
            ypr = phase[j, delay:]
            y   = phase[j, : -delay]
            x   = phase[i, : -delay]

            max_val = max(ypr.max(), y.max(), x.max()) + 1

            # Compute distributions using histogram functions
            # P_y (1D)
            P_y = np.bincount(y, minlength=max_val)

            # P_ypr_y (2D)
            # We use histogram2d with range set based on max_val
            P_ypr_y, _, _ = np.histogram2d(ypr, y, bins=(max_val, max_val),
                                           range=[[0, max_val], [0, max_val]])

            # P_y_x (2D)
            P_y_x, _, _ = np.histogram2d(y, x, bins=(max_val, max_val),
                                         range=[[0, max_val], [0, max_val]])

            # P_ypr_y_x (3D)
            # histogramdd for 3D histogram
            P_ypr_y_x, _ = np.histogramdd((ypr, y, x),
                                          bins=(max_val, max_val, max_val),
                                          range=((0, max_val), (0, max_val), (0, max_val)))

            # Normalize probabilities
            P_y       = P_y / length
            P_ypr_y   = P_ypr_y / length
            P_y_x     = P_y_x / length
            P_ypr_y_x = P_ypr_y_x / length

            # Compute entropies
            # Add eps to avoid log2(0)
            def entropy(P):
                P = P[P > 0]
                return -np.sum(P * np.log2(P + eps))

            Hy         = entropy(P_y)
            Hypr_y     = entropy(P_ypr_y)
            Hy_x       = entropy(P_y_x)
            Hypr_y_x   = entropy(P_ypr_y_x)

            # Compute Partial Transfer Entropy
            PTE[i, j] = Hypr_y + Hy_x - Hy - Hypr_y_x

    return PTE


def compute_dPTE_rawPTE(
    phase: npt.NDArray, delay: int
) -> Tuple[npt.NDArray, npt.NDArray]:
    raw_PTE = compute_PTE(phase, delay)
    tmp = np.triu(raw_PTE) + np.tril(raw_PTE).T
    with np.errstate(divide="ignore", invalid="ignore"):
        dPTE = np.triu(raw_PTE / tmp, 1) + np.tril(raw_PTE / tmp.T, -1)
    return dPTE, raw_PTE

def PTE(d_phase, delay):
    return compute_dPTE_rawPTE(d_phase, delay)



In [6]:
import glob

import os 

import mne 
# -------------------------------------------------------------------
# 1) Gather file paths
# -------------------------------------------------------------------
file_paths = glob.glob('./dataset/sub-*/eeg/*.set')

def get_subject_id(filepath):
    for part in filepath.split(os.sep):
        if 'sub-' in part:
            return int(part.replace('sub-', '').strip())
    return None

# Separate into groups
alz_file_paths = []
ctrl_file_paths = []
ftd_file_paths = []

for fpath in file_paths:
    subj_id = get_subject_id(fpath)
    if subj_id is None:
        continue
    if 1 <= subj_id <= 36:
        alz_file_paths.append(fpath)
    elif 37 <= subj_id <= 65:
        ctrl_file_paths.append(fpath)
    elif subj_id >= 66:
        ftd_file_paths.append(fpath)

# Frequency bands
freq_bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "gamma": (30, 45)
}


# def compute_subject_pte(
#     file_path,
#     freq_bands,
#     n_channels=19,
#     win_length=10.0,  # seconds per window
#     overlap=0.5      # 50% overlap
# ):
#     """
#     For the ENTIRE recording:
#       1. Divide into consecutive 1-minute segments.
#       2. For each 1-minute segment, compute 12 overlapping windows 
#          of 10s each (50% overlap).
#       3. For each of 5 frequency bands and 19 channels, compute 
#          a 19x19 PTE matrix for each window.
#     Returns:
#       subj_id  : Subject ID
#       pte_data : shape (n_minutes, 12, 5, 19, 19)
#     """
#     subj_id = get_subject_id(file_path)
#     if subj_id is None:
#         raise ValueError(f"Could not parse subject ID from {file_path}")

#     # Load entire recording
#     raw = mne.io.read_raw_eeglab(file_path, preload=True)
#     fs = raw.info['sfreq']

#     # Number of samples in 1 minute
#     samples_per_minute = int(60 * fs)
#     # How many full 1-min segments fit in the data
#     data = raw.get_data()  # shape: (n_channels, n_samples_total)
#     n_samples_total = data.shape[1]
#     n_minutes = n_samples_total // samples_per_minute
#     if n_minutes < 1:
#         raise ValueError(
#             "Not enough data for even one 1-minute segment."
#         )

#     # For each 1-min chunk, we have 12 windows of 10s each (50% overlap):
#     # 10s window => step_size = 5s => we get 0,5,10,15,...,55 => 12 windows.
#     n_windows_per_min = 12  # fixed for 60s chunk, 10s window, 50% overlap
#     n_bands = len(freq_bands)

#     # Final array
#     # shape: (n_minutes, 12, 5, 19, 19)
#     pte_data = np.zeros((n_minutes, n_windows_per_min, n_bands, n_channels, n_channels))

#     # We'll compute band-specific Hilbert transforms once for the ENTIRE recording,
#     # then slice out the minute chunks & windows to compute PTE.
#     for b_idx, (band_name, (fmin, fmax)) in enumerate(freq_bands.items()):
#         # Copy & filter
#         raw_band = raw.copy().filter(fmin, fmax)
#         band_data = raw_band.get_data()  # shape: (n_channels, n_samples_total)

#         # Hilbert transform -> phase
#         phase_matrix = np.angle(hilbert(band_data, axis=1))

#         # Discretize
#         delay = get_delay(phase_matrix)
#         binsize = get_binsize(phase_matrix)
#         disc_phase = get_discretized_phase(phase_matrix + np.pi, binsize)

#         # Now process each 1-minute chunk
#         for m_idx in range(n_minutes):
#             min_start = m_idx * samples_per_minute
#             min_end = min_start + samples_per_minute
#             chunk_phase = disc_phase[:, min_start:min_end]
#             # chunk_phase shape: (n_channels, 60s_of_samples)

#             # Slide 12 windows (each 10s = 10 * fs samples, step = 5 * fs)
#             samples_per_window = int(win_length * fs)  # 10s
#             step_size = int(samples_per_window * (1 - overlap))  # 5s
#             w_count = 0
#             for start_sec in range(0, 60 - 10 + 1, 5):  # 0..50..55
#                 start_samp = start_sec * int(fs)
#                 end_samp = start_samp + samples_per_window
#                 # Extract window
#                 window_phase = chunk_phase[:, start_samp:end_samp]  # shape: (19, 10s_of_samples)

#                 # Compute PTE
#                 dPTE_vals, raw_PTE_vals = PTE(window_phase, delay)  # should each be (19, 19)
#                 # or if you only need one of them, pick accordingly

#                 # Store in pte_data
#                 pte_data[m_idx, w_count, b_idx, :, :] = dPTE_vals
#                 w_count += 1

#     return subj_id, pte_data


# def process_and_save_subjects(file_paths, group_label=""):
#     """
#     - For each subject in file_paths:
#       1. compute_subject_pte(...) => shape (n_minutes, 12, 5, 19, 19)
#       2. Save npz with pte_data, subject_id
#     """
#     for fpath in file_paths:
#         subj_id, pte_data = compute_subject_pte(fpath, freq_bands=freq_bands)

#         # e.g. sub-1_PTE_alz.npz
#         out_filename = f"sub-{subj_id}_PTE_{group_label}.npz"
#         print(f"Saving {out_filename} with shape {pte_data.shape}")
#         np.savez(out_filename, pte_data=pte_data, subject_id=subj_id)


# # -------------------------------------------------------------------
# # Finally, run computations for each group
# # -------------------------------------------------------------------
# process_and_save_subjects(alz_file_paths, group_label="alz")
# process_and_save_subjects(ctrl_file_paths, group_label="ctrl")
# process_and_save_subjects(ftd_file_paths, group_label="ftd")

In [11]:
def compute_subject_pte(
    file_path,
    freq_bands,
    n_channels=19,
    win_length=10.0,  # seconds per window
    overlap=0.5      # 50% overlap
):
    """
    For the ENTIRE recording:
      1. Divide into consecutive 1-minute segments.
      2. For each 1-minute segment, compute 11 overlapping windows 
         of 10s each (50% overlap).
      3. For each of 5 frequency bands and 19 channels, compute 
         a 19x19 PTE matrix for each window.

    Returns:
      subj_id  : Subject ID
      pte_data : numpy array, shape (n_minutes, 11, 5, 19, 19)
                 where:
                   - n_minutes = total number of full 1-min segments
                   - 11 = number of windows in each 1-min segment
                   - 5 = number of frequency bands
                   - 19 = number of channels
    """
    subj_id = get_subject_id(file_path)
    if subj_id is None:
        raise ValueError(f"Could not parse subject ID from {file_path}")

    # Load entire recording
    raw = mne.io.read_raw_eeglab(file_path, preload=True)
    fs = raw.info['sfreq']

    # Number of samples in 1 minute
    samples_per_minute = int(60 * fs)

    # How many full 1-min segments fit in the data?
    data = raw.get_data()  # shape: (n_channels, n_samples_total)
    n_samples_total = data.shape[1]
    n_minutes = n_samples_total // samples_per_minute
    if n_minutes < 1:
        raise ValueError(
            "Not enough data for even one 1-minute segment."
        )

    # We want 11 windows in each 1-minute chunk
    # window length = 10s => step_size = 5s => 
    # start times: 0,5,10,15,20,25,30,35,40,45,50 => 11 windows
    n_windows_per_min = 11
    n_bands = len(freq_bands)

    # Final array
    # shape: (n_minutes, 11, 5, 19, 19)
    pte_data = np.zeros((n_minutes, n_windows_per_min, n_bands, n_channels, n_channels))

    # Compute Hilbert transform for each band on the ENTIRE recording
    # then slice for each minute and each window
    for b_idx, (band_name, (fmin, fmax)) in enumerate(freq_bands.items()):
        # Copy & filter
        raw_band = raw.copy().filter(fmin, fmax)
        band_data = raw_band.get_data()  # shape: (n_channels, n_samples_total)

        # Hilbert transform -> instantaneous phase
        phase_matrix = np.angle(hilbert(band_data, axis=1))

        # Discretize
        delay = get_delay(phase_matrix)
        binsize = get_binsize(phase_matrix)
        disc_phase = get_discretized_phase(phase_matrix + np.pi, binsize)

        # Process each 1-minute chunk
        for m_idx in range(n_minutes):
            min_start = m_idx * samples_per_minute
            min_end = min_start + samples_per_minute
            chunk_phase = disc_phase[:, min_start:min_end]
            # shape of chunk_phase: (n_channels, 60s_of_samples)

            # Slide 11 windows in this 1-minute chunk
            samples_per_window = int(win_length * fs)  # 10s
            step_size = int(samples_per_window * (1 - overlap))  # 5s

            # start times = 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50
            w_count = 0
            for start_sec in range(0, 60 - 10 + 1, 5):
                #  range(0, 51, 5) => 0,5,10,15,20,25,30,35,40,45,50 (11 windows)
                start_samp = int(start_sec * fs)
                end_samp = start_samp + samples_per_window

                # Extract window of data
                window_phase = chunk_phase[:, start_samp:end_samp]  # shape: (19, 10s_of_samples)

                # Compute PTE
                dPTE_vals, raw_PTE_vals = PTE(window_phase, delay)  # each (19, 19)

                # Store in pte_data
                pte_data[m_idx, w_count, b_idx, :, :] = dPTE_vals
                w_count += 1

            

    return subj_id, pte_data
# -------------------------------------------------------------------
# 5) Function to process each group and save results
# -------------------------------------------------------------------
def process_and_save_subjects(file_paths, group_label=""):
    """
    - For each subject in file_paths:
      1. compute_subject_pte(...) => shape (n_minutes, 11, 5, 19, 19)
      2. Save npz with pte_data, subject_id
    """
    for fpath in file_paths:
        subj_id, pte_data = compute_subject_pte(fpath, freq_bands=freq_bands)

        # e.g. sub-1_PTE_alz.npz
        out_filename = f"sub-{subj_id}_PTE_{group_label}.npz"
        print(f"Saving {out_filename} with shape {pte_data.shape}")
        np.savez(out_filename, pte_data=pte_data, subject_id=subj_id)

# -------------------------------------------------------------------
# 6) Run computations for each group
# -------------------------------------------------------------------
process_and_save_subjects(alz_file_paths, group_label="alz")
process_and_save_subjects(ctrl_file_paths, group_label="ctrl")
process_and_save_subjects(ftd_file_paths, group_label="ftd")

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


delay =  188
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 825 samples (1.650 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


delay =  43
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 825 samples (1.650 s)



...

delay =  26
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 12 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 551 samples (1.102 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


delay =  13
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 221 samples (0.442 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.2s finished


delay =  7
Saving sub-80_PTE_ftd.npz with shape (15, 11, 5, 19, 19)


In [ ]:
import glob
import os
import numpy as np
import mne

# Make sure you have installed torcheeg:
#    pip install torcheeg
from torcheeg import transforms

# -------------------------------------------------------------------
# 1) Gather file paths
# -------------------------------------------------------------------
file_paths = glob.glob('/teamspace/studios/this_studio/dataset/derivatives/sub-*/eeg/*.set')

def get_subject_id(filepath):
    """
    Parse the subject ID from a file path like:
      ./dataset/derivatives/sub-01/eeg/... 
    This function expects a pattern 'sub-###' in the path.
    """
    for part in filepath.split(os.sep):
        if 'sub-' in part:
            # e.g. 'sub-01' -> '01' -> int(1)
            return int(part.replace('sub-', '').strip())
    return None

# Separate file paths into groups
alz_file_paths = []
ctrl_file_paths = []
ftd_file_paths = []

for fpath in file_paths:
    subj_id = get_subject_id(fpath)
    if subj_id is None:
        continue
    if 1 <= subj_id <= 36:
        alz_file_paths.append(fpath)
    elif 37 <= subj_id <= 65:
        ctrl_file_paths.append(fpath)
    elif subj_id >= 66:
        ftd_file_paths.append(fpath)

# Frequency bands of interest
freq_bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "gamma": (30, 45)
}


def compute_subject_psd(
    file_path,
    freq_bands,
    n_channels=19,
    win_length=10.0,  # seconds per window
    overlap=0.5      # 50% overlap
):

    subj_id = get_subject_id(file_path)
    if subj_id is None:
        raise ValueError(f"Could not parse subject ID from {file_path}")

    # Load entire recording with MNE
    raw = mne.io.read_raw_eeglab(file_path, preload=True)
    from torcheeg import transforms

    t = transforms.BandDifferentialEntropy()
    #make fixed epochs
    epochs = mne.make_fixed_length_epochs(raw, duration=60, preload=True)
    psd_data = np.zeros((len(epochs), n_channels, len(freq_bands)))

    # Loop through all the frequency bands
    for band_idx, (band_name, (fmin, fmax)) in enumerate(freq_bands.items()):
        temp_psd_epochs = epochs.compute_psd(fmin=fmin, fmax=fmax)
        temp_psd = temp_psd_epochs.get_data()
        temp_psd = np.mean(temp_psd, axis=2)  # Average across the frequency dimension
        # print(temp_psd.shape)
        psd_data[:, :, band_idx] = temp_psd  # Use numerical index instead of band name




    return subj_id, psd_data


def process_and_save_subjects(file_paths, group_label=""):
    """
    For each subject in file_paths:
      1) compute_subject_psd(...) => shape (n_minutes, 12, n_channels, n_bands)
      2) Save to .npz with psd_data and subject_id
    """
    for fpath in file_paths:
        subj_id, psd_data = compute_subject_psd(fpath, freq_bands=freq_bands)

        out_filename = f"sub-{subj_id}_PSD_{group_label}.npz"
        print(f"Saving {out_filename} with shape {psd_data.shape}")
        np.savez(f"./psd_features/{out_filename}", psd_data=psd_data, subject_id=subj_id)


# -------------------------------------------------------------------
# Finally, run computations for each group
# -------------------------------------------------------------------
process_and_save_subjects(alz_file_paths,  group_label="alz")
process_and_save_subjects(ctrl_file_paths, group_label="ctrl")
process_and_save_subjects(ftd_file_paths,  group_label="ftd")


In [ ]:
import glob
import os
import numpy as np
import mne

# Make sure you have installed torcheeg:
#    pip install torcheeg
from torcheeg import transforms

# -------------------------------------------------------------------
# 1) Gather file paths
# -------------------------------------------------------------------
file_paths = glob.glob('/teamspace/studios/this_studio/dataset/derivatives/sub-*/eeg/*.set')

def get_subject_id(filepath):
    """
    Parse the subject ID from a file path like:
      ./dataset/derivatives/sub-01/eeg/... 
    This function expects a pattern 'sub-###' in the path.
    """
    for part in filepath.split(os.sep):
        if 'sub-' in part:
            # e.g. 'sub-01' -> '01' -> int(1)
            return int(part.replace('sub-', '').strip())
    return None

# Separate file paths into groups
alz_file_paths = []
ctrl_file_paths = []
ftd_file_paths = []

for fpath in file_paths:
    subj_id = get_subject_id(fpath)
    if subj_id is None:
        continue
    if 1 <= subj_id <= 36:
        alz_file_paths.append(fpath)
    elif 37 <= subj_id <= 65:
        ctrl_file_paths.append(fpath)
    elif subj_id >= 66:
        ftd_file_paths.append(fpath)

# Frequency bands of interest
freq_bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "gamma": (30, 45)
}


def compute_subject_psd(
    file_path,
    freq_bands,
    n_channels=19,
    win_length=10.0,  # seconds per window
    overlap=0.5      # 50% overlap
):

    subj_id = get_subject_id(file_path)
    if subj_id is None:
        raise ValueError(f"Could not parse subject ID from {file_path}")

    # Load entire recording with MNE
    raw = mne.io.read_raw_eeglab(file_path, preload=True)

    #make fixed epochs
    epochs = mne.make_fixed_length_epochs(raw, duration=60, preload=True)
    psd_data = np.zeros((len(epochs), n_channels, len(freq_bands)))

    # Loop through all the frequency bands
    for band_idx, (band_name, (fmin, fmax)) in enumerate(freq_bands.items()):
        temp_psd_epochs = epochs.compute_psd(fmin=fmin, fmax=fmax)
        temp_psd = temp_psd_epochs.get_data() * 1e6  # Convert to microvolts
        temp_psd = np.mean(temp_psd, axis=2)  # Average across the frequency dimension
        # print(temp_psd.shape)
        psd_data[:, :, band_idx] = temp_psd  # Use numerical index instead of band name


    print(psd_data)

    return subj_id, psd_data


def process_and_save_subjects(file_paths, group_label=""):
    """
    For each subject in file_paths:
      1) compute_subject_psd(...) => shape (n_minutes, 12, n_channels, n_bands)
      2) Save to .npz with psd_data and subject_id
    """
    for fpath in file_paths:
        subj_id, psd_data = compute_subject_psd(fpath, freq_bands=freq_bands)

        out_filename = f"sub-{subj_id}_PSD_{group_label}.npz"
        print(f"Saving {out_filename} with shape {psd_data.shape}")
        np.savez(f"./psd_features/{out_filename}", psd_data=psd_data, subject_id=subj_id)


# -------------------------------------------------------------------
# Finally, run computations for each group
# -------------------------------------------------------------------
process_and_save_subjects(alz_file_paths,  group_label="alz")
process_and_save_subjects(ctrl_file_paths, group_label="ctrl")
process_and_save_subjects(ftd_file_paths,  group_label="ftd")
